# Data processing - Level 2

**Not done in this case**
Time averaging:
- 10 sec all instruments, but
- mSEMS remains native)

In [ ]:
%load_ext autoreload
%autoreload 2

## Load level 1.5 dataset

In [ ]:
import logging
from helikite.config import load_config
from helikite.constants import constants

logging.basicConfig(level=constants.LOGLEVEL_CONSOLE)
config = load_config(constants.CONFIG_FILE)

# define your input and output directories here
input_dir = config.processing_dir
output_level2_dir = config.processing_dir / "Level2"
output_level2_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
from helikite.classes.data_processing_level1_5 import DataProcessorLevel1_5
from helikite.metadata.utils import load_parquet

df_level1_5 = DataProcessorLevel1_5.read_data(input_dir / "Level1.5" / f"level1.5_{config.flight_basename}.csv")

_, metadata = load_parquet(input_dir / "Level0" / f"level0_{config.flight_basename}.parquet")

In [ ]:
from helikite.classes.data_processing_level2 import DataProcessorLevel2
from helikite.classes.output_schemas import OutputSchemas

data_processor = DataProcessorLevel2(getattr(OutputSchemas, config.output_schema), df_level1_5, metadata)

## Final file
### 10 second resolution
**Averaging over 10 seconds**

In [ ]:
data_processor.average(rule="10s")

### Flight profile
The same flight profile as for Level 1.5, but with the data averaged over 10 seconds.

Example of customizing the flight profile plot:
```python
import dataclasses

custom_variables = data_processor.output_schema.flight_profile_variables.copy()
custom_variables[4] = dataclasses.replace(custom_variables[4], x_divider=20, x_bounds=(0, 120))

data_processor.plot_flight_profiles(config.flight_basename, save_path, variables=custom_variables)

In [ ]:
save_path = output_level2_dir / f'Level2_{config.flight_basename}_Flight_{config.flight}.png'

data_processor.plot_flight_profiles(config.flight_basename, save_path, variables=None)

###Size distributions

In [ ]:
save_path = output_level2_dir / f'Level2_{config.flight_basename}_SizeDistr_Flight_{config.flight}.png'
data_processor.plot_size_distr(config.flight_basename, save_path, time_start=None, time_end=None)

In [ ]:
data_processor.export_data(output_level2_dir / f"level2_{config.flight_basename}.csv")

**FINISHED**

# Random code bits

### Remove WS and WD outliers after data processing

In [ ]:
import pandas as pd

file_path = output_level2_dir / f"level2_{config.flight_basename}.csv"

flight51 = pd.read_csv(file_path)
flight51

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# set to True to apply the filter on WS and WD data
apply_wind_filter = False

if apply_wind_filter:
    # Replace WindSpeed > 15 or < 0.1 with NaN
    mask = (flight51['WindSpeed'] > 5) | (flight51['WindSpeed'] < 0.1)
    flight51.loc[mask, 'WindSpeed'] = np.nan

    # Replace corresponding WindDir with NaN
    flight51.loc[mask, 'WindDir'] = np.nan

    # Plot WindSpeed vs Altitude
    plt.figure(figsize=(6,8))
    plt.plot(flight51['WindSpeed'], flight51['Altitude'], color='blue', linewidth=2)

    plt.xlabel("Wind Speed [m/s]", fontsize=12, fontweight='bold')
    plt.ylabel("Altitude [m]", fontsize=12, fontweight='bold')
    plt.title("Wind Speed Profile (NaN > 15 or < 0.1) - Flight 51", fontsize=14, fontweight='bold')
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()

In [ ]:
if apply_wind_filter:
    # Save cleaned flight51
    flight51.to_csv(file_path, index=False)

    print(f"Flight 51 saved to {file_path}")

In [ ]:
import pandas as pd

file_path = output_level2_dir / f"level2_{config.flight_basename}.csv"

flight51 = pd.read_csv(file_path)
flight51

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# set to True to apply the filter on WS and WD data
apply_wind_filter = False

if apply_wind_filter:
    # Replace WindSpeed > 15 or < 0.1 with NaN
    mask = (flight51['WindSpeed'] > 5) | (flight51['WindSpeed'] < 0.1)
    flight51.loc[mask, 'WindSpeed'] = np.nan

    # Replace corresponding WindDir with NaN
    flight51.loc[mask, 'WindDir'] = np.nan

    # Plot WindSpeed vs Altitude
    plt.figure(figsize=(6,8))
    plt.plot(flight51['WindSpeed'], flight51['Altitude'], color='blue', linewidth=2)

    plt.xlabel("Wind Speed [m/s]", fontsize=12, fontweight='bold')
    plt.ylabel("Altitude [m]", fontsize=12, fontweight='bold')
    plt.title("Wind Speed Profile (NaN > 15 or < 0.1) - Flight 51", fontsize=14, fontweight='bold')
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()

In [ ]:
if apply_wind_filter:
    # Save cleaned flight51
    flight51.to_csv(file_path, index=False)

    print(f"Flight 51 saved to {file_path}")